In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time

In [7]:
# 크롤링 설정 및 시작
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('/chromedriver',options=options)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [8]:
# URL 접속
driver.get("https://understat.com/league/EPL")

In [9]:
# 시즌 선택
season_select = driver.find_elements_by_xpath("""//*[@id="header"]/div/div[2]/div""")
season_select[0].click()
season_select_num = driver.find_elements_by_xpath("""//*[@id="header"]/div/div[2]/ul/li[8]""")
season_select_num[0].click()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  after removing the cwd from sys.path.


In [10]:
import pandas as pd

month_list = ['Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul']
month_list_temp = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# 날짜 초기값 설정
day = 15
year = 14
index_month = 0

# 빈 DataFrame 선언
df = pd.DataFrame(columns=['Date', 'Team', 'xG', 'xA', 'xPTS', 'match_or_not'])

while(1):
    
    # 날짜 수정
    month = month_list[index_month]
    day = day + 1
    if month == 'Feb':
        if day == 29:
            day = 0
            index_month = index_month + 1
            continue
    
    elif month in ['Sep', 'Nov', 'Feb', 'Apr', 'Jun']:
        if day == 31:
            day = 0
            index_month = index_month + 1
            continue
            
    else:
        if day == 32:
            day = 0
            index_month = index_month + 1
            
            if month == 'Dec':
                year = year + 1
            
            continue
    
    
    date = "{} {}, 20{}".format(month, day, year)
    print(date)
    
    # 날짜 크롤링 및 입력
    time.sleep(1)
    date_start = driver.find_elements_by_xpath("""//*[@class="datepicker hasDatepicker"]""")
    
    date_start[0].clear()
    date_start[0].send_keys(date)
    date_start[0].send_keys(Keys.ENTER)
    date_start[0].send_keys(Keys.ENTER)
    time.sleep(1)

    date_start[1].clear()
    date_start[1].send_keys(date)
    date_start[1].send_keys(Keys.ENTER)
    date_start[1].send_keys(Keys.ENTER)
    time.sleep(1)

    # 치뤄진 경기 없으면 continue
    xpath_match_or_not = '//*[@id="league-chemp"]/table/tbody/tr[1]/td[3]'
    match_or_not_temp = driver.find_elements_by_xpath(xpath_match_or_not)
    time.sleep(1)
    match_or_not_temp = match_or_not_temp[0].text
    if match_or_not_temp == '0':
        continue
    
    
    # 팀 명, 경기 여부 크롤링
    team_list = []
    match_or_not_list = []
    for i in range(1, 21):
        xpath_str = '//*[@id="league-chemp"]/table/tbody/tr[{}]/td[2]/a'.format(i)
        time.sleep(1)
        xpath_match = '//*[@id="league-chemp"]/table/tbody/tr[{}]/td[3]'.format(i)
        time.sleep(1)
        
        team = driver.find_elements_by_xpath(xpath_str)
        match = driver.find_elements_by_xpath(xpath_match)
        team = team[0].text
        match = match[0].text

        team_list.append(team)
        match_or_not_list.append(match)
    
    
    # xG 리스트 크롤링
    xG = driver.find_elements_by_xpath('//div[@id="league-chemp"]//table/tbody//td[@class="align-right nowrap"]')
    time.sleep(1)
    
    # xG, xA, xPTS 전처리
    xG_list, xA_list, xPTS_list = [], [], []
    
    for i in range(len(xG)):
        
        xG_temp = xG[i].text
        
        if '-' in xG_temp:
            index = xG_temp.find('-')
            xG_temp = xG_temp[0:index]
        if '+' in xG_temp:
            index = xG_temp.find('+')
            xG_temp = xG_temp[0:index]

        if i % 3 == 0:
            xG_list.append(xG_temp)
        elif i % 3 == 1:
            xA_list.append(xG_temp)
        else:
            xPTS_list.append(xG_temp)
    
    # 날짜 전처리
    date_temp = date.replace(",", "")
    date_temp = date_temp.split()

    for month in month_list_temp:
        if date_temp[0] == month:
            index = month_list_temp.index(month)+1
            index = str(index).zfill(2)

    date_day = date_temp[1].zfill(2)
    date_pretreat = "{}/{}/{}".format(date_day, index, date_temp[2])

    # 최종 리스트 선언
    df_temp = [[0 for col in range(6)] for row in range(20)]

    for i in range(20):
        df_temp[i][0] = date_pretreat
        df_temp[i][1] = team_list[i]
        df_temp[i][2] = xG_list[i]
        df_temp[i][3] = xA_list[i]
        df_temp[i][4] = xPTS_list[i]
        df_temp[i][5] = match_or_not_list[i]
    
    # 치뤄지지 않은 경기(요소) 삭제
    df_temp_final = df_temp.copy()
    for i in reversed(range(len(df_temp_final))):
        if df_temp_final[i][5] == '0':
            del df_temp_final[i]

    df_temp = pd.DataFrame(df_temp_final, columns=['Date', 'Team', 'xG', 'xA', 'xPTS', 'match_or_not'])
    df = pd.concat([df, df_temp])
    print(df)
    
    if date == "May 25, 2015":
        break;


Aug 16, 2014


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:83: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:84: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:93: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead


          Date                  Team    xG    xA  xPTS match_or_not
0   16/08/2014               Arsenal  1.55  0.16  2.79            1
1   16/08/2014               Swansea  0.28  1.17  0.48            1
2   16/08/2014           Aston Villa  0.91  0.42  1.83            1
3   16/08/2014             Tottenham  1.02  1.85  0.71            1
4   16/08/2014                  Hull  1.12  1.90  0.74            1
5   16/08/2014               Everton  0.61  1.28  0.76            1
6   16/08/2014             Leicester  1.28  0.61  1.95            1
7   16/08/2014  West Bromwich Albion  1.68  0.99  2.04            1
8   16/08/2014            Sunderland  0.99  1.68  0.70            1
9   16/08/2014        Crystal Palace  0.16  1.55  0.11            1
10  16/08/2014     Manchester United  1.17  0.28  2.24            1
11  16/08/2014              West Ham  1.85  1.02  2.02            1
12  16/08/2014                 Stoke  0.42  0.91  0.80            1
13  16/08/2014   Queens Park Rangers  1.90  1.12

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:83: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:84: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:93: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead


          Date                  Team    xG    xA  xPTS match_or_not
0   16/08/2014               Arsenal  1.55  0.16  2.79            1
1   16/08/2014               Swansea  0.28  1.17  0.48            1
2   16/08/2014           Aston Villa  0.91  0.42  1.83            1
3   16/08/2014             Tottenham  1.02  1.85  0.71            1
4   16/08/2014                  Hull  1.12  1.90  0.74            1
5   16/08/2014               Everton  0.61  1.28  0.76            1
6   16/08/2014             Leicester  1.28  0.61  1.95            1
7   16/08/2014  West Bromwich Albion  1.68  0.99  2.04            1
8   16/08/2014            Sunderland  0.99  1.68  0.70            1
9   16/08/2014        Crystal Palace  0.16  1.55  0.11            1
10  16/08/2014     Manchester United  1.17  0.28  2.24            1
11  16/08/2014              West Ham  1.85  1.02  2.02            1
12  16/08/2014                 Stoke  0.42  0.91  0.80            1
13  16/08/2014   Queens Park Rangers  1.90  1.12

...

May 22, 2015
May 23, 2015
May 24, 2015


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:83: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:84: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:93: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead


          Date                  Team    xG    xA  xPTS match_or_not
0   16/08/2014               Arsenal  1.55  0.16  2.79            1
1   16/08/2014               Swansea  0.28  1.17  0.48            1
2   16/08/2014           Aston Villa  0.91  0.42  1.83            1
3   16/08/2014             Tottenham  1.02  1.85  0.71            1
4   16/08/2014                  Hull  1.12  1.90  0.74            1
..         ...                   ...   ...   ...   ...          ...
15  24/05/2015           Southampton  0.78  2.44  0.28            1
16  24/05/2015              West Ham  0.42  1.64  0.42            1
17  24/05/2015  West Bromwich Albion  0.57  3.17  0.11            1
18  24/05/2015   Queens Park Rangers  1.27  2.17  0.77            1
19  24/05/2015             Liverpool  1.02  2.16  0.53            1

[760 rows x 6 columns]
May 25, 2015


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:67: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:83: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:84: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:93: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead


          Date                  Team    xG    xA  xPTS match_or_not
0   16/08/2014               Arsenal  1.55  0.16  2.79            1
1   16/08/2014               Swansea  0.28  1.17  0.48            1
2   16/08/2014           Aston Villa  0.91  0.42  1.83            1
3   16/08/2014             Tottenham  1.02  1.85  0.71            1
4   16/08/2014                  Hull  1.12  1.90  0.74            1
..         ...                   ...   ...   ...   ...          ...
15  25/05/2015           Southampton  0.78  2.44  0.28            1
16  25/05/2015              West Ham  0.42  1.64  0.42            1
17  25/05/2015  West Bromwich Albion  0.57  3.17  0.11            1
18  25/05/2015   Queens Park Rangers  1.27  2.17  0.77            1
19  25/05/2015             Liverpool  1.02  2.16  0.53            1

[780 rows x 6 columns]


In [11]:
df = df.reset_index(drop=True)
df

,Date,Team,xG,xA,xPTS,match_or_not
0,16/08/2014,Arsenal,1.55,0.16,2.79,1
1,16/08/2014,Swansea,0.28,1.17,0.48,1
2,16/08/2014,Aston Villa,0.91,0.42,1.83,1
3,16/08/2014,Tottenham,1.02,1.85,0.71,1
4,16/08/2014,Hull,1.12,1.90,0.74,1
...,...,...,...,...,...,...
775,25/05/2015,Southampton,0.78,2.44,0.28,1
776,25/05/2015,West Ham,0.42,1.64,0.42,1
777,25/05/2015,West Bromwich Albion,0.57,3.17,0.11,1
778,25/05/2015,Queens Park Rangers,1.27,2.17,0.77,1


In [12]:
df.to_csv("xG_2014-15.csv", index=False)